In [126]:
import pandas as pd
import numpy as np
import geopy.distance
from models import *

In [179]:
# Find top 5 locations for a given location of a user

locations = (Locations
             .select(Locations.street, fn.sqrt(fn.pow(51.69-Locations.dis_lat, 2) + 
                                               fn.pow(5.30-Locations.dis_long, 2))
                     .alias('dist'))
             .order_by(SQL('dist'))
             .limit(5)
             .dicts())

# pick one image for each of the top 5 locations
# I CANNOT SUCCEED GR MYRTHE